# Graph-Search algorithm

Este algoritmo es la base para otros más eficientes. Conviene entenderlo bien.
Veamos cómo funciona aplicado al problema de Rumania.

Implementación simple para fines didácticos.


In [1]:
# DEFINICIÓN DEL PROBLEMA DE RUMANIA

# Mapa de conexiones entre ciudades
mapa_rumania = {
    'Arad': ['Zerind', 'Sibiu', 'Timisoara'],
    'Zerind': ['Arad', 'Oradea'],
    'Sibiu': ['Arad', 'Fagaras', 'Rimnicu', 'Oradea'],
    'Timisoara': ['Arad', 'Lugoj'],
    'Oradea': ['Zerind', 'Sibiu'],
    'Fagaras': ['Sibiu', 'Bucharest'],
    'Rimnicu': ['Sibiu', 'Pitesti', 'Craiova'],
    'Lugoj': ['Timisoara', 'Mehadia'],
    'Mehadia': ['Lugoj', 'Drobeta'],
    'Drobeta': ['Mehadia', 'Craiova'],
    'Craiova': ['Drobeta', 'Rimnicu', 'Pitesti'],
    'Pitesti': ['Rimnicu', 'Craiova', 'Bucharest'],
    'Bucharest': ['Fagaras', 'Pitesti', 'Giurgiu', 'Urziceni'],
    'Giurgiu': ['Bucharest'],
    'Urziceni': ['Bucharest', 'Hirsova', 'Vaslui'],
    'Hirsova': ['Urziceni', 'Eforie'],
    'Eforie': ['Hirsova'],
    'Vaslui': ['Urziceni', 'Iasi'],
    'Iasi': ['Vaslui', 'Neamt'],
    'Neamt': ['Iasi']
}

# Distancias entre ciudades (bidireccionales)
distancias_rumania = {
    ('Arad', 'Zerind'): 75, ('Zerind', 'Arad'): 75,
    ('Arad', 'Sibiu'): 140, ('Sibiu', 'Arad'): 140,
    ('Arad', 'Timisoara'): 118, ('Timisoara', 'Arad'): 118,
    ('Zerind', 'Oradea'): 71, ('Oradea', 'Zerind'): 71,
    ('Sibiu', 'Oradea'): 151, ('Oradea', 'Sibiu'): 151,
    ('Sibiu', 'Fagaras'): 99, ('Fagaras', 'Sibiu'): 99,
    ('Sibiu', 'Rimnicu'): 80, ('Rimnicu', 'Sibiu'): 80,
    ('Timisoara', 'Lugoj'): 111, ('Lugoj', 'Timisoara'): 111,
    ('Lugoj', 'Mehadia'): 70, ('Mehadia', 'Lugoj'): 70,
    ('Mehadia', 'Drobeta'): 75, ('Drobeta', 'Mehadia'): 75,
    ('Drobeta', 'Craiova'): 120, ('Craiova', 'Drobeta'): 120,
    ('Rimnicu', 'Craiova'): 146, ('Craiova', 'Rimnicu'): 146,
    ('Rimnicu', 'Pitesti'): 97, ('Pitesti', 'Rimnicu'): 97,
    ('Craiova', 'Pitesti'): 138, ('Pitesti', 'Craiova'): 138,
    ('Fagaras', 'Bucharest'): 211, ('Bucharest', 'Fagaras'): 211,
    ('Pitesti', 'Bucharest'): 101, ('Bucharest', 'Pitesti'): 101,
    ('Bucharest', 'Giurgiu'): 90, ('Giurgiu', 'Bucharest'): 90,
    ('Bucharest', 'Urziceni'): 85, ('Urziceni', 'Bucharest'): 85,
    ('Urziceni', 'Hirsova'): 98, ('Hirsova', 'Urziceni'): 98,
    ('Urziceni', 'Vaslui'): 142, ('Vaslui', 'Urziceni'): 142,
    ('Hirsova', 'Eforie'): 86, ('Eforie', 'Hirsova'): 86,
    ('Vaslui', 'Iasi'): 92, ('Iasi', 'Vaslui'): 92,
    ('Iasi', 'Neamt'): 87, ('Neamt', 'Iasi'): 87
}



In [2]:
def graph_search(mapa, distancias, estado_inicial, estado_objetivo):
    """
    Implementación del algoritmo Graph-Search genérico

    Args:
        mapa: diccionario con las conexiones entre ciudades
        distancias: diccionario con las distancias entre ciudades
        estado_inicial: ciudad de inicio
        estado_objetivo: ciudad objetivo

    Returns:
        tuple: (camino_encontrado, costo_total, estadisticas)
    """

    # Inicializar la frontera con el estado inicial
    # Cada nodo en la frontera es una tupla: (estado_actual, camino_hasta_aqui, costo_acumulado)
    frontera = [(estado_inicial, [estado_inicial], 0)] # <----- IMPORTANTE

    # Conjunto de estados ya explorados (visited)
    explorados = set()                                 # <----- IMPORTANTE

    # Estadísticas para análisis
    nodos_expandidos = 0
    paso = 0

    print("=== INICIO DEL ALGORITMO  ===")
    print(f"Estado inicial: {estado_inicial}")
    print(f"Estado objetivo: {estado_objetivo}")
    print("-" * 50)

    # LOOP PRINCIPAL
    while True:
        paso += 1
        print(f"\n--- PASO {paso} ---")

        # Verificar si la frontera está vacía
        if not frontera:
            print(" FAIL!!: La frontera está vacía. No hay solución.")
            return None, float('inf'), {"nodos_expandidos": nodos_expandidos, "pasos": paso}

        print(f"Frontera actual: {len(frontera)} nodos")
        for i, (estado, camino, costo) in enumerate(frontera):
            print(f"  {i+1}. {estado} | Camino: {' -> '.join(camino)} | Costo: {costo}")

        # Elegir nodo hoja y removerlo de la frontera (FIFO - como BFS)
        nodo_actual = frontera.pop(0)                  # <----- IMPORTANTE
        estado_actual, camino_actual, costo_actual = nodo_actual

        print(f"\n Nodo elegido: {estado_actual}")
        print(f"   Camino hasta aquí: {' -> '.join(camino_actual)}")
        print(f"   Costo acumulado: {costo_actual}")

        # Verificar si ya exploramos este estado (para evitar ciclos)
        if estado_actual in explorados:
            print(f"  {estado_actual} ya fue explorado. Saltando...")
            continue

        # Agregar el estado actual a explorados
        explorados.add(estado_actual)                 # <----- IMPORTANTE
        print(f" Agregando {estado_actual} a explorados")

        # TEST DE OBJETIVO
        if estado_actual == estado_objetivo:
            print(f"\n🎯 ¡OBJETIVO ENCONTRADO!")
            print(f"   Camino solución: {' -> '.join(camino_actual)}")
            print(f"   Costo total: {costo_actual}")
            return camino_actual, costo_actual, {"nodos_expandidos": nodos_expandidos, "pasos": paso}

        # EXPANDIR NODO
        nodos_expandidos += 1
        print(f"\n🔄 Expandiendo {estado_actual}...")

        # Obtener acciones posibles (ciudades conectadas)
        ciudades_conectadas = mapa.get(estado_actual, []) # <----- IMPORTANTE (obtenemos posibles acciones del nodo actual)
        print(f"   Ciudades conectadas: {ciudades_conectadas}")

        # Generar nodos sucesores
        sucesores_agregados = 0
        for ciudad_destino in ciudades_conectadas:
            # Verificar si ya exploramos este estado.      # <----- IMPORTANTE (para no generar loops)
            if ciudad_destino not in explorados:
                # Calcular nuevo camino y costo
                nuevo_camino = camino_actual + [ciudad_destino]
                costo_viaje = distancias.get((estado_actual, ciudad_destino), float('inf'))
                nuevo_costo = costo_actual + costo_viaje

                # Verificar si ya está en la frontera (opcional: mantener el mejor costo)
                ya_en_frontera = False
                for i, (estado_f, _, costo_f) in enumerate(frontera):
                    if estado_f == ciudad_destino:
                        if nuevo_costo < costo_f:
                            # Reemplazar con mejor costo
                            frontera[i] = (ciudad_destino, nuevo_camino, nuevo_costo)
                            print(f"   ↻ Actualizando {ciudad_destino} en frontera (mejor costo: {nuevo_costo})")
                        ya_en_frontera = True
                        break

                if not ya_en_frontera:                    # <----- IMPORTANTE (si al expandor, nodo no está en frontera, lo agregamos)
                    # Agregar nuevo nodo a la frontera
                    frontera.append((ciudad_destino, nuevo_camino, nuevo_costo))
                    sucesores_agregados += 1
                    print(f"    Agregando {ciudad_destino} (costo: {costo_viaje}, total: {nuevo_costo})")
            else:
                print(f"   {ciudad_destino} ya explorado, saltando")

        print(f"   Sucesores agregados: {sucesores_agregados}")
        print(f"   Estados explorados hasta ahora: {sorted(list(explorados))}")



In [4]:
# EJECUCIÓN DEL ALGORITMO



In [3]:
# Ejecutar el algoritmo
resultado = graph_search(
      mapa=mapa_rumania,
      distancias=distancias_rumania,
      estado_inicial='Arad',
      estado_objetivo='Bucharest'
)

=== INICIO DEL ALGORITMO  ===
Estado inicial: Arad
Estado objetivo: Bucharest
--------------------------------------------------

--- PASO 1 ---
Frontera actual: 1 nodos
  1. Arad | Camino: Arad | Costo: 0

 Nodo elegido: Arad
   Camino hasta aquí: Arad
   Costo acumulado: 0
 Agregando Arad a explorados

🔄 Expandiendo Arad...
   Ciudades conectadas: ['Zerind', 'Sibiu', 'Timisoara']
    Agregando Zerind (costo: 75, total: 75)
    Agregando Sibiu (costo: 140, total: 140)
    Agregando Timisoara (costo: 118, total: 118)
   Sucesores agregados: 3
   Estados explorados hasta ahora: ['Arad']

--- PASO 2 ---
Frontera actual: 3 nodos
  1. Zerind | Camino: Arad -> Zerind | Costo: 75
  2. Sibiu | Camino: Arad -> Sibiu | Costo: 140
  3. Timisoara | Camino: Arad -> Timisoara | Costo: 118

 Nodo elegido: Zerind
   Camino hasta aquí: Arad -> Zerind
   Costo acumulado: 75
 Agregando Zerind a explorados

🔄 Expandiendo Zerind...
   Ciudades conectadas: ['Arad', 'Oradea']
   Arad ya explorado, saltando
